Group members:
Sima Esmaeili (3766857)
Behrooz Montazeran (3769073)
Mikail Deniz Cayoglu (3437201)

# Exercise 1

Assuming that all machines are functioning independently:
a) 
If the probability of failing is p, then the probability of working is (1 - p).
The probability that all will keep working is (1 - p) ^n.
thus, the probability that at least one will fail is 1 - (1 - p) ^n

b) 
The probability that exactly k machines will fail is n nCr k ·p^k·(1 - p)^(n- k)

c) 
p1+p2+…+pn=\sum_{k=0}^{n}{\binom{n}{k}p^k{(1-p)}^{n-k}}-\binom{n}{0}p^0{(1-p)}^{n-0}={(p+(1-p))}^{n-0}-{(1-p)}^n=
1-{(1-p)}^n


# Exercise 2

a)
The “join” transformation can help us join two pairs of RDDs based on their key.
The “groupBy” transformation will group the data in the original RDD. It creates a set of key value pairs, where the key is output of a user function, and the value is all items for which the function yields this key.
the sortByKey() function on a dataset of (K, V) pairs, the data is sorted according to the key K in another RDD.


b)

Transformer: union()

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from IPython.display import clear_output
import pyspark.sql.types as stypes
import os
from pyspark.sql import functions as F
from math import ceil

In [2]:
_conf = SparkConf()
_conf.set('spark.driver.host', '127.0.0.1')
_conf.set('log.level', 'ERROR')
_cpu_count = os.cpu_count()
sc = SparkContext(master=f"local[{_cpu_count}]",
                  appName="Exercise 01",
                  conf=_conf).getOrCreate()
sc.setLogLevel('ERROR')
clear_output()
spark = SparkSession(sc)

In [3]:
data1 = [('Item1', 1),('Item2', 1)]
data2 = [('Item3', 1),('Item4', 1)]
rdd1=spark.sparkContext.parallelize(data1)
rdd2=spark.sparkContext.parallelize(data2)
rdd3 =rdd1.union(rdd2)
for element in rdd3.collect():
 print(element)

('Item1', 1)
('Item2', 1)
('Item3', 1)
('Item4', 1)


Transformer: reduceByKey()

In [4]:
data = [('Item1', 1),('Item2', 1),('Item3', 1),('Item1', 1)]
rdd=spark.sparkContext.parallelize(data)
rdd2=rdd.reduceByKey(lambda a,b: a+b)
for element in rdd2.collect():
 print(element)

('Item3', 1)
('Item1', 2)
('Item2', 1)


Transformer: sortByKey()

In [5]:
data = [('Item1', 1),('Item2', 1),('Item3', 1),('Item1', 1),('Item3', 1),('Item3', 1)]
rdd=spark.sparkContext.parallelize(data)
rdd2=rdd.reduceByKey(lambda a,b: a+b)
rdd3 = rdd2.map(lambda x: (x[1],x[0])).sortByKey()
for element in rdd3.collect():
 print(element)

(1, 'Item2')
(2, 'Item1')
(3, 'Item3')


Action: collect

In [6]:
listRdd = spark.sparkContext.parallelize([1,2,3,4,5,6,7])
data = listRdd.collect()
print(data)

[1, 2, 3, 4, 5, 6, 7]


Action: first

In [7]:
print("first :  "+str(listRdd.first()))

first :  1


Action: max

In [8]:
print("max :  "+str(listRdd.max()))

max :  7


# Exercise 3a

Broadcast will send a shared variable (such as hashing table) to all nodes and they will cache it. 
This mechanism is actually a pointer to that value. In this case there is no need to send a copy of whole datasets to all 
machines to implement them or to all tasks separately. It reduces the cost of network communication. Tasks process 
the mapping and connect the data to the hashing table that comes from the broadcasting. Nodes are the holder of distributed 
datasets, which will process tasks.

# Exercise 3b

Accumulator aggregate multiple values at the same time, which can be done by reduce function too but in it you have to send back all of the values together in one structure. In video he explains a filter example in which we want to skip bad records and bas bytes with the help of accumulator. To define a custom accumulator, we create a class and an implicit object in it and the rest of functions in it.
Accumulator mechanism:
Accumulators are variables that are only “added” to through an associative operation and can therefore, be efficiently supported in parallel. They can be used to implement counters (as in MapReduce) or sums. Spark natively supports accumulators of numeric types, and programmers can add support for new types.
In reduce function:
•  each partition is processed sequentially element by element 
•  multiple partitions can be processed at the same time either by a single worker (multiple executor threads) or different workers 
•  partial results are fetched to the driver where the final reduction is applied

# Exercise 3c

partitionBy(), join(), reducedByKey(). Partitioning will use the benefit of network bandwidth by controlling the partitioning of RDD contents across nodes. By use of the partitioning (the datasets across the machines partitioned) and sending the ranks to the machine that has the corresponding links for that hash code. Spark did it by:
each RDD has an optional partitioner object, any shuffle operation on an RDD with a partitioner will respect that partitioner, any shuffle operation on two RDDs will take on the partitioner of one of them, if one is set, otherwise by default use hashpartitioner. 
To make a custom partitioner :by making a class of extending the partitioner, in which some method such as number of partitions, getting the partition and equal should be made.


Exercise 4a
----------------

In [9]:

_dataset_path = os.path.abspath(os.path.join(os.getcwd(), 'adult/adult.test'))
adult_dataset = sc.textFile(_dataset_path)

In [10]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
          'marital_status', 'occupation', 'relationship',
          'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
          'native_country', 'income']
# faulty entries
pieces = adult_dataset.map(lambda row: [y.strip() for y in x] if len(x := row.split(',')) == 15 
                           else [None for x in range(15)])
df = pieces.toDF(columns)
# filter out the faulty entries
df = df.filter(df["age"] != 'NULL')
df.printSchema()
df.show()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: string (nullable = true)
 |-- capital_loss: string (nullable = true)
 |-- hours_per_week: string (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)

+---+----------------+------+------------+-------------+------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|   education|education_num|    marital_status|       occupation| relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|na

In [11]:
columns_to_remove = ['fnlwgt', 'occupation', 'relationship', 'race']
df = df.drop(*columns_to_remove)
df.printSchema()
df.show()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: string (nullable = true)
 |-- capital_loss: string (nullable = true)
 |-- hours_per_week: string (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)

+---+----------------+------------+-------------+------------------+------+------------+------------+--------------+--------------+------+
|age|       workclass|   education|education_num|    marital_status|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+----------------+------------+-------------+------------------+------+------------+------------+--------------+--------------+------+
| 25|         Private|        11th|            7|     Never-married|  Male|           0|           0|            40|

Exercise 4b
----------------


In [14]:
male_df = df.select(df["marital_status"], df["sex"]
                   ).filter(df["sex"] == "Male").groupBy('marital_status').count(
).withColumnRenamed("count", "male_count")

female_df = df.select(df["marital_status"], df["sex"]
                   ).filter(df["sex"] == "Female").groupBy('marital_status').count(
).withColumnRenamed("count", "female_count")

joined_df = male_df.join(female_df, 'marital_status')
joined_df = joined_df.withColumn('male_ratio_percentage', F.ceil((F.col('male_count') 
                                                / (F.col('male_count') + F.col('female_count'))
                                               ) * 100))
joined_df.show()

+--------------------+----------+------------+---------------------+
|      marital_status|male_count|female_count|male_ratio_percentage|
+--------------------+----------+------------+---------------------+
|           Separated|       205|         300|                   41|
|       Never-married|      2983|        2451|                   55|
|Married-spouse-ab...|       111|          99|                   53|
|            Divorced|       861|        1329|                   40|
|             Widowed|       117|         408|                   23|
|   Married-AF-spouse|         3|          11|                   22|
|  Married-civ-spouse|      6580|         823|                   89|
+--------------------+----------+------------+---------------------+



Exercise 4c
----------------

In [17]:
df_4c = df.filter(
    df["sex"] == 'Female'
).filter(
    df["income"] == ">50K."
).select(
    df["hours_per_week"], df["native_country"]
)
# df_4c = df_4c.groupBy("native_country").count()
# df_4c = df_4c.agg(F.mean("hours_per_week"), F.count("native_country"))
# df_4c = df_4c.groupBy("native_country").count()
df_4c.show()

+--------------+--------------+
|hours_per_week|native_country|
+--------------+--------------+
|            36| United-States|
|            10| United-States|
|            10| United-States|
|            40| United-States|
|            40| United-States|
|            38| United-States|
|            50| United-States|
|            50| United-States|
|            40| United-States|
|            45| United-States|
|            50| United-States|
|            40| United-States|
|            12| United-States|
|            35| United-States|
|            50| United-States|
|            24| United-States|
|            40| United-States|
|            40| United-States|
|            10| United-States|
|            40| United-States|
+--------------+--------------+
only showing top 20 rows

